# Use-case : Drugs having a putative effect on a given metabolism

## Goal

Given a metabolic network that could be loaded as a cobra model (eq. in sbml format for instance), derive a list of genes that regulates directly the network's metabolic reactions (later, this list is refered as Targets).
Then, this list is used to find the Drugs having a direct impact on the metabolism (from the KeggDrugs repository).
pyBravo is then used to reconstruct the upstream gene regulatory network of the Targets genes. This provides to a bigger list of direct and undirect regulators (this list is refered as CompleteTargets) which allows to find the Drugs having a non direct impact on the metabolism.

Notice that when the metabolic network is provided with a functioning objective function, a single reaction deletion analysis is first performed to reduce the size Targets gene list and focus on the genes having an effect on the objective value.

## Part 1 : installing the required libraries (not required if the environment is prepared in advance)

In [1]:
!pip3 install cobra
!pip3 install python-libsbml

## Part 2 : importing the required python libraries

In [2]:
import cobra

## Part 3 : Parameters

In [3]:
Param_SBMLfile = "../models/iLiverCancer1715.xml"
#Param_Percentage = 1.1 # Complete Target list (does not require any objective function)
Param_Percentage = 0.95 # Restriction of the gene such that, once deleted, reduce the objective value for more that 5%

filenameCorrespondanceEnsemblGenenames = "../tables/Correspondance_Ensembl_genename.txt"
filenameCorrespondanceEnsemblEnzymes = "../tables/Correspondance_Enzymes_genenames.txt"
filenameCorrespondanceEnsemblTransporters = "../tables/Correspondance_Transporters_genenames.txt"
filenameCorrespondanceHMREnsembl = "../tables/Correspondance_HMR_Ensembl.csv"


## Part 4 : reading the association tables
HMR -> EnsemBl id for generegulators

EnsemBl id -> gene names

....

In [4]:
filename = filenameCorrespondanceEnsemblGenenames

EnsemblToGeneNames = dict()
GeneNamesToEnsembl = dict()

EnsemblToGeneNamesList = dict()
GeneNamesToEnsemblList = dict()


fd = open(filename,"r")

lines = fd.readlines()
for l in lines:
    t=l.split(";")
    t[2]=t[2].replace("\n","")
    EnsemblToGeneNames[t[0]]=t[2]
    GeneNamesToEnsembl[t[2]]=t[0]
    if (not EnsemblToGeneNamesList.get(t[0])):
        EnsemblToGeneNamesList[t[0]]=list()
    if (not GeneNamesToEnsemblList.get(t[2])):
        GeneNamesToEnsemblList[t[2]]=list()
    EnsemblToGeneNamesList[t[0]].append(t[2])
    GeneNamesToEnsemblList[t[2]].append(t[0])
    
fd.close()

# Important Missing genename associations
GeneNamesToEnsembl["TGIF"]=GeneNamesToEnsembl["TGIF1"]
GeneNamesToEnsembl["CART1"]=GeneNamesToEnsembl["ALX1"]
EnsemblToGeneNames["ENSG00000122718"]="OR2S2"
EnsemblToGeneNames["ENSG00000155640"] = "C10orf12"
GeneNamesToEnsembl["C10orf12"]="ENSG00000155640"
EnsemblToGeneNames["ENSG00000168260"] = "C14orf183"
GeneNamesToEnsembl["C14orf183"]="ENSG00000168260"
EnsemblToGeneNames["ENSG00000168787"] = "OR12D2"
EnsemblToGeneNames["ENSG00000170803"] = "OR2AG1"
EnsemblToGeneNames["ENSG00000171484"] = "OR1B1"
EnsemblToGeneNames["ENSG00000172381"] = "OR6Q1"




# Tables 3 et 4 : Correspondance (Enzymes,Transporters) <-> Ensembl

filename = filenameCorrespondanceEnsemblEnzymes

ECTCDBToEnsembl = dict()
fd = open(filename,"r")

lines = fd.readlines()
for l in lines:
    t=l.split(";")
    L=t[1].replace("\n","").split(",")
    ECTCDBToEnsembl[t[0]]=list()
    for g in L:
        if (GeneNamesToEnsembl.get(g)):
            ECTCDBToEnsembl[t[0]].append(GeneNamesToEnsembl.get(g))
fd.close()


filename = filenameCorrespondanceEnsemblTransporters

fd = open(filename,"r")

lines = fd.readlines()
for l in lines:
    t=l.split(";")
    L=t[1].replace("\n","").split(",")
    ECTCDBToEnsembl[t[0]]=list()
    for g in L:
        if (GeneNamesToEnsembl.get(g)):
            ECTCDBToEnsembl[t[0]].append(GeneNamesToEnsembl.get(g))
fd.close()


# Table 7 : HMR to Ensembl correspondance

fd = open(filenameCorrespondanceHMREnsembl,"r")

lines=fd.readlines()

HMR_to_Ensembl = dict()
for g in lines:
    t=g.split(";")
    t[1]=(t[1].replace("\n",""))
    HMR_to_Ensembl[t[0]]=t[1].split("|")
    
fd.close()








# ????? Utilité ?

for g in EnsemblToGeneNamesList:
    EnsemblToGeneNamesList[g]=list(set(EnsemblToGeneNamesList.get(g)))
    EnsemblToGeneNamesList.get(g).sort()
    if (len(EnsemblToGeneNamesList.get(g))>1):
        print(g,EnsemblToGeneNamesList.get(g).sort())


# In[15]:

import json, io

# Read JSON file
with open('../tables/Correspondance_Ensembl_GeneNames_by_SPARQL.json') as data_file:
    genelist = json.load(data_file)


GenenameToEnsembl=dict()
for gid in genelist:
    genename=genelist[gid][0]
    if not (GenenameToEnsembl.get(genename)):
        GenenameToEnsembl[genename]=list()
        GenenameToEnsembl[genename].append(gid)

GeneAltNameToName=dict()
for gid in genelist:
    genename=genelist[gid][0]
    GeneAltNameToName[genename]=genename
    for i in range(1,len(genelist[gid])):
        GeneAltNameToName[genelist[gid][i]]=genename



In [5]:
## Part 5 : reading the cobra model

In [6]:
model = cobra.io.read_sbml_model(Param_SBMLfile)

# Required for MetabolicAtlas models : set the objective function
model.reactions.CancerBiomass_OF.objective_coefficient=1

Model does not contain SBML fbc package information.
SBML package 'layout' not supported by cobrapy, information is not parsed
SBML package 'render' not supported by cobrapy, information is not parsed
urn:miriam:lipidmaps:LMFA01030283 does not conform to 'http(s)://identifiers.org/collection/id' or'http(s)://identifiers.org/COLLECTION:id
Use of FORMULA in the notes element is discouraged, use fbc:chemicalFormula instead: <Species M_m00003c "(10Z)-heptadecenoic acid">
urn:miriam:lipidmaps:LMFA01030283 does not conform to 'http(s)://identifiers.org/collection/id' or'http(s)://identifiers.org/COLLECTION:id
Use of FORMULA in the notes element is discouraged, use fbc:chemicalFormula instead: <Species M_m00003r "(10Z)-heptadecenoic acid">
urn:miriam:lipidmaps:LMFA01030283 does not conform to 'http(s)://identifiers.org/collection/id' or'http(s)://identifiers.org/COLLECTION:id
Use of FORMULA in the notes element is discouraged, use fbc:chemicalFormula instead: <Species M_m00003s "(10Z)-heptade

urn:miriam:lipidmaps:LMFA01030367 does not conform to 'http(s)://identifiers.org/collection/id' or'http(s)://identifiers.org/COLLECTION:id
Use of FORMULA in the notes element is discouraged, use fbc:chemicalFormula instead: <Species M_m00017s "(13Z)-eicosenoic acid">
Use of FORMULA in the notes element is discouraged, use fbc:chemicalFormula instead: <Species M_m00018c "(13Z)-eicosenoyl-CoA">
Use of FORMULA in the notes element is discouraged, use fbc:chemicalFormula instead: <Species M_m00018m "(13Z)-eicosenoyl-CoA">
Use of FORMULA in the notes element is discouraged, use fbc:chemicalFormula instead: <Species M_m00018p "(13Z)-eicosenoyl-CoA">
Use of FORMULA in the notes element is discouraged, use fbc:chemicalFormula instead: <Species M_m00018r "(13Z)-eicosenoyl-CoA">
urn:miriam:lipidmaps:LMFA01030290 does not conform to 'http(s)://identifiers.org/collection/id' or'http(s)://identifiers.org/COLLECTION:id
Use of FORMULA in the notes element is discouraged, use fbc:chemicalFormula inste

Use of FORMULA in the notes element is discouraged, use fbc:chemicalFormula instead: <Species M_m00038c "(2E)-decenoyl-[ACP]">
urn:miriam:obo.chebi:CHEBI:10723 does not conform to 'http(s)://identifiers.org/collection/id' or'http(s)://identifiers.org/COLLECTION:id
Use of FORMULA in the notes element is discouraged, use fbc:chemicalFormula instead: <Species M_m00039m "(2E)-decenoyl-CoA">
urn:miriam:obo.chebi:CHEBI:10723 does not conform to 'http(s)://identifiers.org/collection/id' or'http(s)://identifiers.org/COLLECTION:id
Use of FORMULA in the notes element is discouraged, use fbc:chemicalFormula instead: <Species M_m00039p "(2E)-decenoyl-CoA">
Use of FORMULA in the notes element is discouraged, use fbc:chemicalFormula instead: <Species M_m00040c "(2E)-docosenoyl-CoA">
Use of FORMULA in the notes element is discouraged, use fbc:chemicalFormula instead: <Species M_m00040m "(2E)-docosenoyl-CoA">
Use of FORMULA in the notes element is discouraged, use fbc:chemicalFormula instead: <Species

urn:miriam:obo.chebi:CHEBI:27721 does not conform to 'http(s)://identifiers.org/collection/id' or'http(s)://identifiers.org/COLLECTION:id
Use of FORMULA in the notes element is discouraged, use fbc:chemicalFormula instead: <Species M_m00066p "(2E)-tetradecenoyl-CoA">
Use of FORMULA in the notes element is discouraged, use fbc:chemicalFormula instead: <Species M_m00067c "(2E)-tricosenoyl-CoA">
Use of FORMULA in the notes element is discouraged, use fbc:chemicalFormula instead: <Species M_m00068c "(2E)-tridecenoyl-[ACP]">
Use of FORMULA in the notes element is discouraged, use fbc:chemicalFormula instead: <Species M_m00069m "(2E)-tridecenoyl-CoA">
Use of FORMULA in the notes element is discouraged, use fbc:chemicalFormula instead: <Species M_m00070c "(2E)-undecenoyl-[ACP]">
Use of FORMULA in the notes element is discouraged, use fbc:chemicalFormula instead: <Species M_m00071m "(2E)-undecenoyl-CoA">
Use of FORMULA in the notes element is discouraged, use fbc:chemicalFormula instead: <Spec

urn:miriam:obo.chebi:CHEBI:65132 does not conform to 'http(s)://identifiers.org/collection/id' or'http(s)://identifiers.org/COLLECTION:id
Use of FORMULA in the notes element is discouraged, use fbc:chemicalFormula instead: <Species M_m00095r "(4Z,7Z,10Z,13Z,16Z,19Z)-docosahexaenoyl-CoA">
urn:miriam:kegg.compound:C03740 does not conform to 'http(s)://identifiers.org/collection/id' or'http(s)://identifiers.org/COLLECTION:id
Use of FORMULA in the notes element is discouraged, use fbc:chemicalFormula instead: <Species M_m00097s "(5-L-glutamyl)-L-amino acid">
Use of FORMULA in the notes element is discouraged, use fbc:chemicalFormula instead: <Species M_m00099m "(5Z)-dodecenoyl-CoA">
Use of FORMULA in the notes element is discouraged, use fbc:chemicalFormula instead: <Species M_m00099p "(5Z)-dodecenoyl-CoA">
Use of FORMULA in the notes element is discouraged, use fbc:chemicalFormula instead: <Species M_m00100c "(5Z,8Z,11Z)-eicosatrienoylcarnitine">
Use of FORMULA in the notes element is dis

...

Missing upper flux bound set to '1000.0' for  reaction: '<Reaction R_HMR_9165 "oxalate[x]">'
Missing lower flux bound set to '-1000.0' for  reaction: '<Reaction R_HMR_9166 "chitin-component">'
Missing upper flux bound set to '1000.0' for  reaction: '<Reaction R_HMR_9166 "chitin-component">'
Missing lower flux bound set to '-1000.0' for  reaction: '<Reaction R_HMR_9168 "glucosamine">'
Missing upper flux bound set to '1000.0' for  reaction: '<Reaction R_HMR_9168 "glucosamine">'
Missing lower flux bound set to '-1000.0' for  reaction: '<Reaction R_HMR_9169 "selenate">'
Missing upper flux bound set to '1000.0' for  reaction: '<Reaction R_HMR_9169 "selenate">'
Missing lower flux bound set to '-1000.0' for  reaction: '<Reaction R_HMR_9171 "2-acetyl-1-alkyl-sn-glycero-3-phosphocholine">'
Missing upper flux bound set to '1000.0' for  reaction: '<Reaction R_HMR_9171 "2-acetyl-1-alkyl-sn-glycero-3-phosphocholine">'
Missing lower flux bound set to '-1000.0' for  reaction: '<Reaction R_HMR_9172 "1

Missing upper flux bound set to '1000.0' for  reaction: '<Reaction R_HMR_9283 "glycochenodeoxycholate[x]">'
Missing lower flux bound set to '-1000.0' for  reaction: '<Reaction R_HMR_9284 "taurochenodeoxycholate">'
Missing upper flux bound set to '1000.0' for  reaction: '<Reaction R_HMR_9284 "taurochenodeoxycholate">'
Missing lower flux bound set to '-1000.0' for  reaction: '<Reaction R_HMR_9285 "cholesterol">'
Missing upper flux bound set to '1000.0' for  reaction: '<Reaction R_HMR_9285 "cholesterol">'
Missing lower flux bound set to '-1000.0' for  reaction: '<Reaction R_HMR_9286 "citrate[x]">'
Missing upper flux bound set to '1000.0' for  reaction: '<Reaction R_HMR_9286 "citrate[x]">'
Missing lower flux bound set to '-1000.0' for  reaction: '<Reaction R_HMR_9287 "CMP[x]">'
Missing upper flux bound set to '1000.0' for  reaction: '<Reaction R_HMR_9287 "CMP[x]">'
Missing lower flux bound set to '-1000.0' for  reaction: '<Reaction R_HMR_9290 "creatine[x]">'
Missing upper flux bound set to

Missing lower flux bound set to '-1000.0' for  reaction: '<Reaction R_HMR_9396 "prostaglandin E1">'
Missing upper flux bound set to '1000.0' for  reaction: '<Reaction R_HMR_9396 "prostaglandin E1">'
Missing lower flux bound set to '-1000.0' for  reaction: '<Reaction R_HMR_9397 "prostaglandin E2">'
Missing upper flux bound set to '1000.0' for  reaction: '<Reaction R_HMR_9397 "prostaglandin E2">'
Missing lower flux bound set to '-1000.0' for  reaction: '<Reaction R_HMR_9398 "prostaglandin F2alpha">'
Missing upper flux bound set to '1000.0' for  reaction: '<Reaction R_HMR_9398 "prostaglandin F2alpha">'
Missing lower flux bound set to '-1000.0' for  reaction: '<Reaction R_HMR_9404 "retinoate">'
Missing upper flux bound set to '1000.0' for  reaction: '<Reaction R_HMR_9404 "retinoate">'
Missing lower flux bound set to '-1000.0' for  reaction: '<Reaction R_HMR_9407 "D-serine">'
Missing upper flux bound set to '1000.0' for  reaction: '<Reaction R_HMR_9407 "D-serine">'
Missing lower flux bound 

Missing upper flux bound set to '1000.0' for  reaction: '<Reaction R_newExRxn_5 "4-hydroxybenzoate[c]">'
Missing lower flux bound set to '-1000.0' for  reaction: '<Reaction R_newExRxn_6 "trans-4-hydroxy-L-proline[c]">'
Missing upper flux bound set to '1000.0' for  reaction: '<Reaction R_newExRxn_6 "trans-4-hydroxy-L-proline[c]">'
Missing lower flux bound set to '-1000.0' for  reaction: '<Reaction R_newExRxn_7 "5-hydroxyindoleacetate[c]">'
Missing upper flux bound set to '1000.0' for  reaction: '<Reaction R_newExRxn_7 "5-hydroxyindoleacetate[c]">'
Missing lower flux bound set to '-1000.0' for  reaction: '<Reaction R_newExRxn_8 "SAH[c]">'
Missing upper flux bound set to '1000.0' for  reaction: '<Reaction R_newExRxn_8 "SAH[c]">'
Missing lower flux bound set to '-1000.0' for  reaction: '<Reaction R_newExRxn_9 "anthranilate[c]">'
Missing upper flux bound set to '1000.0' for  reaction: '<Reaction R_newExRxn_9 "anthranilate[c]">'
Missing lower flux bound set to '-1000.0' for  reaction: '<Reac

Missing upper flux bound set to '1000.0' for  reaction: '<Reaction R_HMR_8218>'
No objective coefficients in model. Unclear what should be optimized
Missing flux bounds on reactions set to default bounds.As best practise and to avoid confusion flux bounds should be set explicitly on all reactions.


## Part 6 : Performing a single_reaction_deletion analysis [facultative step] 

In [7]:
if Param_Percentage < 1 :
    # rates = cobra.flux_analysis.single_reaction_deletion(model)
    # or naive ré-implementation of the srd analysis if the cobra function fails
    
    rates = dict()
    for r in model.reactions:
        meml = r.lower_bound
        memu = r.upper_bound
        r.lower_bound = 0
        r.upper_bound = 0
        rates[r.id] = model.optimize().objective_value
        r.lower_bound = meml
        r.upper_bound = memu
else:
    rates = dict()
    for r in model.reactions:
        rates[r.id] = 0
        


## Part 7 : Get the regulators (Target gene list)

In [9]:
# Objective reference value
if Param_Percentage < 1 :
    fba = model.optimize().objective_value
else:
    fba = 1
    
def getRegulators(rates, percentage=1.1):
    regulators = dict()
    cpt=0
    for r in rates:
        if (rates.get(r)<percentage*fba):
            if (HMR_to_Ensembl.get(r)!=''):
                cpt=cpt+1
                l=HMR_to_Ensembl.get(r)
                if (l):
                    for i in l:
                        if (i != ''):
                            regulators[i]=1
    return regulators

# Targets (Ensembl Ids)
regulators = getRegulators(rates, Param_Percentage)

TargetGenesEnsembl=list()
for ec in regulators:
    TargetGenesEnsembl+=[ec.replace("'","%27")]
    
TargetGenes = [EnsemblToGeneNames.get(x) for x in TargetGenesEnsembl]

# alternative

"""
TargetGenes=list()
for x in TargetGenesEnsembl:
    if (genelist.get(x)):
        TargetGenes.append(genelist.get(x)[0])
    else:
        print('Missing ',x)
"""

TargetGenes=list(set(TargetGenes))
TargetGenesEnsembl=list(set(TargetGenesEnsembl))
    
print("Target gene list size =",len(TargetGenes))

Target gene list size = 40


In [ ]:
TargetGenes